**참고문헌**  
\[1\] [16-01 트랜스포머(Transformer)](https://wikidocs.net/31379)  
\[2\] [The Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html#prelims)  

## 일반적인 trainning 과정

``` python
model = Model(options)
model.cuda()
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr = options.LR)

for epoch in  range(numEpochs):
    for sampleIndex, sample in enumerate(data_iterator): 
        optimizer.zero_grad()

        outputs = model(sample.input)
        loss = compute_loass(output, sample.gt)
        
        #  
        loss.backward()
        optimizer.step()
        save_weight()
    
```

**LabelSmoothing**이 `loss`을 계산함.
- 정답데이터(ground truth)에 LabelSmoothing을 수행함.
- ground truth는 아마도 인덱스 데이터 인듯
- padding 데이터에는 masking
- 아마도 예측 데이터와 gt로 KLDivLoss 계산
- 출력은 아마도 loss.backward()하는데 사용되어야함.

**NoamOpt**을 이해해봄
- Optimizer를 래핑한 클래스임.
- step() 메서드도 구현됨.
- 학습률 계산을 계산하는 rate() 메서드가 구현됨.

back propagation하는 방식을 이해 해봄.
-------------------------------------

## Greedy Decoding

``` python
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), 
                              y.contiguous().view(-1)) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss * norm
```

``` python
# Train the simple copy task.
V = 11
criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
model = make_model(V, V, N=2)
model_opt = NoamOpt(model.src_embed[0].d_model, 1, 400,
        torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

for epoch in range(10):
    model.train()
    run_epoch(data_gen(V, 30, 20), model, 
              SimpleLossCompute(model.generator, criterion, model_opt))
    model.eval()
    print(run_epoch(data_gen(V, 30, 5), model, 
                    SimpleLossCompute(model.generator, criterion, None)))
```

``` python
def run_epoch(data_iter, model, loss_compute):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.trg, 
                            batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens
```

### Understading hwo to get loss and back propagation

1. **loss**의 계산은 `run_epoch`에서 수행됨  
- `LabelSmoothing`과 `SimpleLossCompute` 클래스가 사용됨.
- run_epoch는 먼저 `SimpleLossCompute` 클래스를 이용함.
- 아래에 관련 코드가 있다.
  
---
``` python
loss = loss_compute(out, batch.trg_y, batch.ntokens) 
```
---

2. 이 부분을 먼저 보자.  
예) paramers:   
`out:` shape -> (8,9,11)  
`batch.trg_y:` shape -> (8,9)  
'batch.ntokens`: 8x9=72  (pad를 제외한 토큰의 수)  

- 이함수는 SimpleLossCompute::\_\_call\_\_ 메서드를 호출함.  
- `criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)`로 loss를 계산함.  
  여기서 batch.trg_y를 out의 shape크기로 변현한후 LabelSmoothing으로 처리함.  
        gt의 shape 변환 : (8,9) -> (8,9,11)  -> (72,11)   
   **padding**는 LabelSmoothing이 적용되지 않음.  
   transformer의 디코더 출력과 gt로  KLDivLoss을 구한 후 평균 loss를 구함.
- `criterion`는 nn.Module를 상속 받아, forward()가 자동으로 호출됨.
---

3. `SimpleLossCompute'에서는 loss계산후에  backpropagation을 수행함.  

``` python
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()

```

4. loss계산에 특이한 부분이 있음.
   - loss계산할 때는 평균을 내고 backward()를 수행하고
   - 리턴할 때는 다시 rescale을 함.
     ``` python
          return loss * norm
     ```
   - run_epoch 함수내에서 loss의 토탈평균을 다시 내고 있음.
     ``` python
        total_loss += loss
        total_tokens += batch.ntokens
        ...
        return total_loss / total_tokens
     ```
     

다음은 harvard transformer에 나온 Greedy Decoding 예제이다. 

``` python
# Train the simple copy task.
V = 11
criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
model = make_model(V, V, N=2)
model_opt = NoamOpt(model.src_embed[0].d_model, 1, 400,
        torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
for epoch in range(1):
    model.train()
    run_epoch(data_gen(V, 8, 1), model, 
              SimpleLossCompute(model.generator, criterion, model_opt))
```